In [117]:
def probAttr(data,attr,val):
    Total=data.shape[0]
    cnt = len(data[data[attr] == val])
    return cnt,cnt/Total

In [118]:
def train(data,Attr,conceptVals,concept): 
    conceptProbs = {} #P(A)
    countConcept={}
    for cVal in conceptVals:
        countConcept[cVal],conceptProbs[cVal] = probAttr(data,concept,cVal)
    
    AttrConcept = {} #P(X/A)
    probability_list = {} #P(X)
    for att in Attr:
        probability_list[att] = {}
        AttrConcept[att] = {}
        for val in Attr[att]:
            AttrConcept[att][val] = {}
            a,probability_list[att][val] = probAttr(data,att,val)
            for cVal in conceptVals:
                dataTemp = data[data[att]==val]
                AttrConcept[att][val][cVal] = len(dataTemp[dataTemp[concept] == cVal])/countConcept[cVal]
            
    print("P(A) : ",conceptProbs,"\n")
    print("P(X/A) : ",AttrConcept,"\n")
    print("P(X) : ",probability_list,"\n")
    return conceptProbs,AttrConcept,probability_list

In [119]:
def test(examples,Attr,concept_list,conceptProbs,AttrConcept,probability_list):
    misclassification_count=0
    Total = len(examples)
    for ex in examples:
        px={}
        for a in Attr:
            for x in ex:
                for c in concept_list:
                    if x in AttrConcept[a]:
                        if c not in px:
                            px[c] = conceptProbs[c]*AttrConcept[a][x][c]/probability_list[a][x]
                        else:
                            px[c] = px[c]*AttrConcept[a][x][c]/probability_list[a][x]
        print(px)
        classification = max(px,key=px.get)
        print("Classification :",classification,"Expected :",ex[-1])
        if(classification!=ex[-1]):
            misclassification_count+=1
    misclassification_rate=misclassification_count*100/Total
    accuracy=100-misclassification_rate
    print("Misclassification Count={}".format(misclassification_count))
    print("Misclassification Rate={}%".format(misclassification_rate))
    print("Accuracy={}%".format(accuracy))

In [120]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    data = DataFrame.from_csv('data_train1.csv')
    concept=str(list(data)[-1])
    concept_list = set(data[concept])
    Attr={}
    for a in list(data)[:-1]:    #Get attribute values
        Attr[a] = set(data[a])
    conceptProbs,AttrConcept,probability_list = train(data,Attr,concept_list,concept)

    examples = DataFrame.from_csv('data_test1.csv')
    test(examples.values,Attr,concept_list,conceptProbs,AttrConcept,probability_list)

In [121]:
main()

P(A) :  {'Yes': 0.6428571428571429, 'No': 0.35714285714285715} 

P(X/A) :  {'Outlook': {'Sunny': {'Yes': 0.2222222222222222, 'No': 0.6}, 'Rain': {'Yes': 0.3333333333333333, 'No': 0.4}, 'Overcast': {'Yes': 0.4444444444444444, 'No': 0.0}}, 'Temperature': {'Mild': {'Yes': 0.4444444444444444, 'No': 0.4}, 'Hot': {'Yes': 0.2222222222222222, 'No': 0.4}, 'Cool': {'Yes': 0.3333333333333333, 'No': 0.2}}, 'Humidity': {'Normal': {'Yes': 0.6666666666666666, 'No': 0.2}, 'High': {'Yes': 0.3333333333333333, 'No': 0.8}}, 'Wind': {'Weak': {'Yes': 0.6666666666666666, 'No': 0.4}, 'Strong': {'Yes': 0.3333333333333333, 'No': 0.6}}} 

P(X) :  {'Outlook': {'Sunny': 0.35714285714285715, 'Rain': 0.35714285714285715, 'Overcast': 0.2857142857142857}, 'Temperature': {'Mild': 0.42857142857142855, 'Hot': 0.2857142857142857, 'Cool': 0.2857142857142857}, 'Humidity': {'Normal': 0.5, 'High': 0.5}, 'Wind': {'Weak': 0.5714285714285714, 'Strong': 0.42857142857142855}} 

{'Yes': 0.24197530864197522, 'No': 0.9408000000000001